In [45]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
import numpy as np
import tensorflow_datasets as tfds

In [46]:
train_ds, test_ds = tfds.load('FashionMNIST', split = ['train', 'test'], as_supervised = True)


2021-11-27 10:48:43.586626: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2021-11-27 10:48:43.592438: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [47]:
def convert_img(image, label):
    image = tf.reshape(image, (1,28,28,1))
    image = tf.cast(image, tf.float32)
    image = image / 255.0
    

    
    label = tf.cast(label, tf.uint8)
    label = tf.one_hot(label, depth=10)
    return image, label

def preprocess_data(dataset):
    dataset = dataset.shuffle(256)
    dataset = dataset.batch(64)
    dataset = dataset.map(convert_img, num_parallel_calls=4)
    dataset = dataset.prefetch(16)
    return dataset

train_ds = train_ds.apply(preprocess_data)
test_ds = test_ds.apply(preprocess_data)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [48]:
input_shape = (16,16)
n_outputs = 128
images, labels = train_ds
    

# instantiate Conv2D layer with 128 filters with kernel size (16,16), without extra padding
convlayer = tf.keras.layers.Conv2D(filters=n_outputs, 
                                       kernel_size=input_shape, 
                                       strides=(1, 1), 
                                       padding="valid")
# instantiate dense layer with 128 outputs
denselayer= tf.keras.layers.Dense(n_outputs)
    
# flatten input to process it with dense layer
flattened_input = tf.keras.layers.Flatten()(image)
denseoutput = denselayer(flattened_input)
    
# process input with convlayer
convoutput = convlayer(image)
    
# reshape weights from dense layer into shape of conv layer weights such that we can use the same weights for both
dense_weights = tf.reshape(denselayer.weights[0], 
           shape = convlayer.weights[0].shape)
    
dense_bias = tf.reshape(denselayer.weights[1],
                           shape= convlayer.weights[1].shape)
    
# assign weights from dense layer to conv layer to show they result in the same output
convlayer.weights[0].assign(dense_weights)
convlayer.weights[1].assign(dense_bias)
    
convoutput = convlayer(image)
    
convoutput = tf.reshape(convoutput, shape = denseoutput.shape)

InvalidArgumentError: Incompatible shapes at component 0: expected [1,28,28,1] but got [64,28,28,1]. [Op:IteratorGetNext]

In [42]:
def train_step(model, input, target, loss_function, optimizer):
    with tf.GradientTape() as tape:
        prediction = model(input)
        loss = loss_function(target, prediction)
        gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

def test(model, test_data, loss_function):
    
    test_accuracy_aggregator = []
    test_loss_aggregator = []
    
    for (input, target) in test_data:
        prediction = model(input)
        sample_test_loss = loss_function(target, prediction)
        sample_test_accuracy = np.argmax(target, axis=1) == np.argmax(prediction, axis=1)
        sample_test_accuracy = np.mean(sample_test_accuracy)
        test_loss_aggregator.append(sample_test_loss.numpy())
        test_accuracy_aggregator.append(np.mean(sample_test_accuracy))
        
    test_loss = tf.reduce_mean(test_loss_aggregator)
    test_accuracy = tf.reduce_mean(test_accuracy_aggregator)
    
    return test_loss, test_accuracy



In [31]:
class MyModel(tf.keras.Model):
    
    def __init__(self):
        super(MyModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(256, activation=tf.nn.sigmoid)
        self.dense2 = tf.keras.layers.Dense(256, activation=tf.nn.sigmoid)
        self.dense3 = tf.keras.layers.Dense(10, activation=tf.nn.softmax)
        
    @tf.function
    def call(self, inputs):
        print(inputs)
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        return x
        
class SimpleDense(tf.keras.layers.Layer):
    
    def __init__(self, units=256):
        super(SimpleDense, self).__init__()
        self.units = units
        self.acitvation = tf.nn.softmax
        
    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units), initializer='random_normal', trainable=True)
        self.b = self.add_weight(shape=(self.units,), initializer='random_normal', trainable=True)
        
    def call(self, inputs): 
        x = tf.matmul(inputs, self.w) + self.b
        x = self.activation(x)
        return x



In [32]:
tf.keras.backend.clear_session()
train_dataset = train_ds.take(1000)
test_dataset = test_ds.take(100)

num_epochs = 10
learning_rate = 0.1

model = MyModel()

cross_entropy_loss = tf.keras.losses.CategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam(learning_rate)

train_losses = []
test_losses = []
test_accuracies = []

test_loss, test_accuracy = test(model, train_dataset, cross_entropy_loss)
test_losses.append(test_loss)
test_accuracies.append(test_accuracy)

train_loss, _ = test(model, train_dataset, cross_entropy_loss)
train_losses.append(train_loss)

for epoch in range(num_epochs):
    print(f'Epoch: {str(epoch)} starting with accuracy {test_accuracies[-1]}')

    epoch_loss_agg = []
    for input,target in train_dataset:
        train_loss = train_step(model, input, target, cross_entropy_loss, optimizer)
        epoch_loss_agg.append(train_loss)
    
    train_losses.append(tf.reduce_mean(epoch_loss_agg))

    test_loss, test_accuracy = test(model, test_dataset, cross_entropy_loss)
    test_losses.append(test_loss)

InvalidArgumentError: Incompatible shapes at component 0: expected [1,28,28,1] but got [64,28,28,1]. [Op:IteratorGetNext]